# Imports

In [ ]:
import pandas as pd, numpy as np
import plotly.express as px
from plotly.graph_objs import Figure
from pathlib import Path
from importlib import reload
import software.analysis as a
reload(a)

# Load Paths

In [ ]:
import sys
sys.path.append(str(Path("../../../").resolve()))
from software.paths import paths

In [ ]:
local_analysis = Path("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/")

In [ ]:
df = pd.read_csv(local_analysis / "20230611-combined.csv", index_col=0)

## Add reverse TC score so that it can be ranked ascending

In [ ]:
df["TanimotoCombo_R"] = 2-df.TanimotoCombo

# Functions and Variables

## Variables

In [ ]:
tc = "TanimotoCombo"
tc_title = "TanimotoCombo Cutoff for Inclusion of Reference Structures"
tcr = "TanimotoCombo_R"
y="Fraction"
posit_r = "POSIT_R"
posit_method="POSIT_Method"
color="Version"
id_col="Compound_ID"
rmsd="RMSD"
method_split=[posit_method]
n=1
good=2
tc_cutoffs = np.linspace(0,2,50)
tcr_cutoffs = np.linspace(2,0,50)
sort_cols = [rmsd, posit_r, "Chemgauss4", tcr]
dates = df.Structure_Date.unique()
date_col = "Structure_Date"
date_title = "Date for Inclusion of Reference Structures"
reference_col = "Structure_Source"
split_cols=["Version"]
sort_col_name="Sorted_By"
full_split_cols=["Version", posit_method]
split_column_sets={"General":split_cols, "Detailed":full_split_cols}
general_split_cols = {"General":split_cols}
detailed_split_cols = {"Detailed":full_split_cols}
frac_title=f"Fraction of Poses < {good}Å from Reference"

## Calculation Functions

## Plotting Functions

### plot kwargs

In [ ]:
df.Version.unique()

In [ ]:
df.POSIT_Method.unique()

In [ ]:
# this doesn't actually work
full_versions = [("All", method) for method in df.POSIT_Method.unique()] + [("Hybrid-Only", "HYBRID")]
full_version_labels = [f"{version}: {method}" for version, method in full_versions]
full_version_label_dict = {og: label for og, label in zip(full_versions, full_version_labels)}

In [ ]:
basic_plot_kwargs = dict(color=color, 
                         )

In [ ]:
big_plot_kwargs = dict(facet_col=sort_col_name,
                         facet_row="Split", 
                         height=600, 
                         width=1200, )

In [ ]:
single_plot_kwargs = dict(height=400, width=600)

In [ ]:
tc_plot_kwargs = dict(x=tc,  
                      labels={tc: tc_title},
                     range_x=[-0.1,2.1],)

In [ ]:
date_plot_kwargs = dict(x=date_col,
                        labels={date_col:date_title},
                       )

In [ ]:
fraction_plot_kwargs = dict(range_y=[-0.1,1.1])

In [ ]:
stats_kwargs = dict(y="Value", error_y="STD")

In [ ]:
def combine_labels_kwargs(list_of_kwargs):
    new_dict = {}
    for kwargs in list_of_kwargs:
        for k,v in kwargs.items():
            if k in new_dict.keys():
                if isinstance(v, dict):
                    for ik, iv in v.items():
                        new_dict[k][ik] = iv
                else:
                    raise NotImplementedError(f"combining these kwargs will not work due to repeated use of {k}")
            else:
                new_dict[k] = v
    return new_dict
    

In [ ]:
combine_labels_kwargs([big_plot_kwargs, tc_plot_kwargs])

In [ ]:
general_posit_kwargs = {sort_col_name: posit_r, "Split":"General"}

### cleanup functions

In [ ]:
def replace_xaxis_labels(fig: Figure, axis_title):
    fig.for_each_xaxis(lambda x: x.update(title = ''))
    fig.add_annotation(x=0.5,y=-0.15,
                   text=axis_title, textangle=0,
                       font=dict(size=16),
                    xref="paper", yref="paper",
            showarrow=False,)
    return fig

In [ ]:
def replace_yaxis_labels(fig: Figure, axis_title):
    fig.for_each_yaxis(lambda y: y.update(title = ''))
    fig.add_annotation(x=-0.05,y=0.5,
                   text=axis_title, textangle=-90,
                       font=dict(size=16),
                    xref="paper", yref="paper",
            showarrow=False,)
    return fig

In [ ]:
def clean_labels(fig):
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    return fig

### scatterplot wrapper

In [ ]:
def scatter_wrapper(df, kwarg_dict, 
                    x_axis_title=None, 
                    y_axis_title=None, 
                    replace_xaxis=False,
                    replace_y_axis=False,
                    clean=True,
                    x_axis_reversed=False
                   ):
    fig:Figure = px.scatter(df, **kwarg_dict, hover_data=df.columns)
    if x_axis_title:
        if replace_xaxis:
            fig = replace_xaxis_labels(fig, x_axis_title)
        else:
            fig.update_xaxes(title=x_axis_title)
    
    if y_axis_title:
        if replace_y_axis:
            fig = replace_yaxis_labels(fig, y_axis_title)
        else:
            fig.update_yaxes(title=y_axis_title)
    
    if clean:
        fig = clean_labels(fig)
    if x_axis_reversed:
        fig.update_xaxes(autorange="reversed")
    return fig

### splify df

In [ ]:
def simplify_df(df, condition_dict):
    new_df = df.copy()
    for column, value in condition_dict.items():
        new_df = new_df[new_df[column] == value]
    return new_df

# Overall Analysis

In [ ]:
df.groupby("Version").nunique()[["Complex_ID", "Compound_ID", "Structure_Source"]]

# Drop self docking results and results with diff compounds

# filter by hybrid-only ones

In [ ]:
cmpds = df[df["Version"] == "Hybrid-Only"].Compound_ID.unique()

In [ ]:
clean = df[df.Compound_ID.isin(cmpds)]

In [ ]:
structures = clean[clean["Version"] == "Hybrid-Only"].Structure_Source.unique()

In [ ]:
clean = clean[clean.Structure_Source.isin(structures)]

## remove self-docked

In [ ]:
clean = clean[clean.Compound_ID != clean.Reference_Ligand]

In [ ]:
clean.groupby("Version").nunique()

## remove failed hybrid-only ones

### skipping for now

In [ ]:
#complexes = clean[clean["Version"] == "Hybrid-Only"].Complex_ID.unique()

In [ ]:
#clean = clean[clean.Complex_ID.isin(complexes)]

In [ ]:
#clean.groupby("Version").nunique()[["Complex_ID", "Compound_ID", "Structure_Source"]]

In [ ]:
clean_all = clean[clean.Version == "All"]

In [ ]:
clean_all.groupby("Version").nunique()[["Complex_ID", "Compound_ID", "Structure_Source"]]

# By Random Sort

In [ ]:
from tqdm import tqdm

In [ ]:
structures = clean.Structure_Source.unique()
random_dfs = []
for i in tqdm(range(0,100)):
    shuffled_structures = np.random.choice(structures, len(structures))
    # print(structures)
    # print(shuffled_structures)
    split_cols_list = []
    score_list = []
    cutoff_list = []
    perc_mols_list = []
    n_references = []
    for n_refs in range(1,len(structures), 5):
        # get a random set of structures
        sampled_structures = shuffled_structures[0:n_refs]
        df_subset = clean[clean.Structure_Source.isin(sampled_structures)]

        # now sort by posit score
        best = df_subset.sort_values("POSIT", ascending=False).groupby(["Version", "Compound_ID"]).head(1)

        # print(best.groupby("Version").nunique())
        perc_good = a.calc_perc_good(best, 
                         score_column=rmsd, 
                         good_score=good,
                         total_mol=len(df_subset.Compound_ID.unique()))
        for split_col in perc_good.index:
            split_cols_list.append(split_col)
            score_list.append(perc_good[split_col])
            n_references.append(n_refs)

    return_df = pd.DataFrame(
        {
            "Fraction": score_list,
            "Version": split_cols_list,
            "Number of References": n_references,
        }
    )
    random_dfs.append(return_df)

In [ ]:
combined = pd.concat(random_dfs)

In [ ]:
stats = combined.groupby(["Version", "Number of References"]).describe().reset_index()

In [ ]:
stats.columns = ["Version", "Number of References", "count", "mean", "std", "min", "25%", "50%", "75%", "max"]

In [ ]:
stats

In [ ]:
fig = scatter_wrapper(stats, kwarg_dict=dict(y="mean", x="Number of References", error_y="std", **basic_plot_kwargs, **single_plot_kwargs, **fraction_plot_kwargs))
fig.update_yaxes(title=frac_title)
fig.update_xaxes(title="Number of References")
fig.show()
fig.write_image("20231002_random_posit_full_all.png")

## use new calculation function

In [ ]:
reload(a)
random_stats = a.calculate_rmsd_stats(clean, query_mol_id="Compound_ID", reference_selection="random", ref_structure_stride=10, score_column="POSIT_R", group_by=["Version"], n_bootstraps=100)
date_stats = a.calculate_rmsd_stats(clean, query_mol_id="Compound_ID", reference_selection="Structure_Date", ref_structure_stride=10, score_column="POSIT_R", group_by=["Version"], n_bootstraps=100)

In [ ]:
tc_stats = a.calculate_rmsd_stats(clean, query_mol_id="Compound_ID", reference_selection="TanimotoCombo_R", ref_structure_stride=10, score_column="POSIT_R", group_by=["Version"], n_bootstraps=100)

In [ ]:
tcr_stats = a.calculate_rmsd_stats(clean, query_mol_id="Compound_ID", reference_selection="TanimotoCombo", ref_structure_stride=10, score_column="POSIT_R", group_by=["Version"], n_bootstraps=100)

In [ ]:
all_stats = pd.concat([random_stats, date_stats, tc_stats, tcr_stats])

In [ ]:
aggregated = all_stats.groupby(["Version", "Number of References", "Structure_Split", "Split_Value_min", "Split_Value_max"]).mean().reset_index()
aggregated["Max"] = all_stats.groupby(["Version", "Number of References", "Structure_Split"]).quantile(0.975).reset_index()["Fraction"] - aggregated["Fraction"]
aggregated["Min"] = aggregated["Fraction"] - all_stats.groupby(["Version", "Number of References", "Structure_Split"]).quantile(0.025).reset_index()["Fraction"]

In [ ]:
fig = scatter_wrapper(aggregated[aggregated["Version"] == "Hybrid-Only"], 
                      dict(
                          y="Fraction", color="Structure_Split", facet_col="Version",
                          color_discrete_sequence=px.colors.qualitative.Dark24,
                          error_y="Max", 
                          error_y_minus="Min",
                          template="seaborn",
                           **fraction_plot_kwargs,
                           x="Number of References", 
                          height=400,
                          width=600
                           # **single_plot_kwargs
                      ),
                     y_axis_title=frac_title,
                     x_axis_title="Number of References"
                     )
fig.show()
fig.write_image("20231002_structure_splits_hybrid.png")

In [ ]:
fig = scatter_wrapper(aggregated, 
                      dict(
                          y="Fraction", color="Structure_Split", facet_col="Version",
                          color_discrete_sequence=px.colors.qualitative.Dark24,
                          error_y="Max", 
                          error_y_minus="Min",
                          template="seaborn",
                           **fraction_plot_kwargs,
                           x="Number of References", 
                          height=600,
                          width=800
                           # **single_plot_kwargs
                      ),
                     y_axis_title=frac_title,
                     x_axis_title="Number of References"
                     )
fig.for_each_yaxis(lambda y: y.update(title = ''))
# fig.add_annotation(x=-0.03, y=0.5,text=frac_title, textangle=-90,
#                     xref="paper", yref="paper")
# fig.update_yaxes(title=frac_title)
fig.update_layout(yaxis1=dict(title=frac_title), height=400, width=800)
fig.show()
fig.write_image("20231002_structure_splits_big.png")

# Test getting traunches of structures

In [ ]:
reload(a)
traunch_random_stats = a.calculate_rmsd_stats(clean, query_mol_id="Compound_ID", reference_selection="random", ref_structure_stride=10, score_column="POSIT_R", group_by=["Version"], n_bootstraps=3, cumulative=False, count_nrefs=True)
traunch_date_stats = a.calculate_rmsd_stats(clean, query_mol_id="Compound_ID", reference_selection="Structure_Date", ref_structure_stride=10, score_column="POSIT_R", group_by=["Version"], n_bootstraps=3, cumulative=False, count_nrefs=True)

In [ ]:
traunch_tc_stats = a.calculate_rmsd_stats(clean, query_mol_id="Compound_ID", reference_selection="TanimotoCombo_R", ref_structure_stride=10, score_column="POSIT_R", group_by=["Version"], n_bootstraps=3, cumulative=False, count_nrefs=True)

In [ ]:
traunch_tcr_stats = a.calculate_rmsd_stats(clean, query_mol_id="Compound_ID", reference_selection="TanimotoCombo", ref_structure_stride=10, score_column="POSIT_R", group_by=["Version"], n_bootstraps=3, cumulative=False, count_nrefs=True)

In [ ]:
traunch_all_stats = pd.concat([traunch_random_stats, traunch_date_stats, traunch_tc_stats, traunch_tcr_stats])

In [ ]:
traunch_all_stats

In [ ]:
traunch_aggregated = traunch_all_stats.groupby(["Version", "Number of References", "Structure_Split", "Split_Value_min", "Split_Value_max", "Mean Number of References", "Max Number of References", "Min Number of References"]).mean().reset_index()
traunch_aggregated["Max"] = traunch_all_stats.groupby(["Version", "Number of References", "Structure_Split"]).quantile(0.975).reset_index()["Fraction"] - traunch_aggregated["Fraction"]
traunch_aggregated["Min"] = traunch_aggregated["Fraction"] - traunch_all_stats.groupby(["Version", "Number of References", "Structure_Split"]).quantile(0.025).reset_index()["Fraction"]

In [ ]:
traunch_aggregated

In [ ]:
n_docked = clean[clean["Version"] == "Hybrid-Only"].groupby("Compound_ID").count()["Docked_File"]

In [ ]:
n_docked.sum()

In [ ]:
fig = px.histogram(n_docked, height=600, width=600)
fig.update_layout(title="Distribution of Successful Docking Runs")
fig.update_xaxes(title="Number of Successful Docked Runs")
fig.show()
fig.write_image("20231010_distribution_successful_docking_runs.png")

In [ ]:
traunch_aggregated["Range"] = [f"{i}-->{j}" for i,j in zip(traunch_aggregated["Number of References"], traunch_aggregated["Number of References"] + 10)]

In [ ]:
traunch_aggregated

In [ ]:
fig = scatter_wrapper(traunch_aggregated, 
                      dict(
                          y="Mean Number of References", color="Version",
                          color_discrete_sequence=px.colors.qualitative.Dark24,
                          template="seaborn",
                           # **fraction_plot_kwargs,
                           x="Range", 
                          height=400,
                          width=600
                           # **single_plot_kwargs
                      ),
                     # y_axis_title=frac_title,
                     x_axis_title="Number of References"
                     )
fig.show()


In [ ]:
fig = scatter_wrapper(traunch_aggregated[traunch_aggregated["Version"] == "Hybrid-Only"], 
                      dict(
                          y="Min Number of References", color="Structure_Split", facet_col="Version",
                          color_discrete_sequence=px.colors.qualitative.Dark24,
                          template="seaborn",
                           # **fraction_plot_kwargs,
                           x="Range", 
                          height=400,
                          width=600
                           # **single_plot_kwargs
                      ),
                     # y_axis_title=frac_title,
                     x_axis_title="Number of References"
                     )
fig.show()

In [ ]:
fig = scatter_wrapper(aggregated[aggregated["Version"] == "Hybrid-Only"], 
                      dict(
                          y="Mean Number of References", color="Structure_Split", facet_col="Version",
                          color_discrete_sequence=px.colors.qualitative.Dark24,
                          template="seaborn",
                           # **fraction_plot_kwargs,
                           x="Range", 
                          height=400,
                          width=600
                           # **single_plot_kwargs
                      ),
                     y_axis_title=frac_title,
                     x_axis_title="Number of References"
                     )
fig.show()

In [ ]:
fig = scatter_wrapper(aggregated, 
                      dict(
                          y="Fraction", color="Structure_Split", facet_col="Version",
                          color_discrete_sequence=px.colors.qualitative.Dark24,
                          error_y="Max", 
                          error_y_minus="Min",
                          template="seaborn",
                           **fraction_plot_kwargs,
                           x="Range", 
                          height=600,
                          width=800
                           # **single_plot_kwargs
                      ),
                     y_axis_title=frac_title,
                     x_axis_title="Number of References"
                     )
fig.for_each_yaxis(lambda y: y.update(title = ''))
# fig.add_annotation(x=-0.03, y=0.5,text=frac_title, textangle=-90,
#                     xref="paper", yref="paper")
# fig.update_yaxes(title=frac_title)
fig.update_layout(yaxis1=dict(title=frac_title), height=400, width=800)
fig.show()


## Using the Kneed Package to 

In [ ]:
from kneed import KneeLocator

In [ ]:
x1 = stats[stats["Version"] == "All"]["Number of References"].unique()

In [ ]:
x2 = stats[stats["Version"] == "Hybrid-Only"]["Number of References"].unique()

In [ ]:
x1 == x2

### convert hybrid_only points to x,y

In [ ]:
x = stats["Number of References"].unique()
for version in stats.Version.unique():
    y = stats[stats["Version"] == version]["mean"].to_list()
    kneedle = KneeLocator(x, y, S=1.0, curve="concave", direction="increasing")
    kneedle.plot_knee_normalized(title=f"{version} Difference Plot")
    print(version, kneedle.knee)